In [6]:
import hashlib

def compute_hash(path, algorithm):
    h = hashlib.new(algorithm)
    with open(path, "rb") as f:
        h.update(f.read())
    return h.hexdigest()

sample = r"C:\Users\aisha\Downloads\ProcessMonitor\Procmon.exe"

print("MD5:   ", compute_hash(sample, "md5"))
print("SHA1:  ", compute_hash(sample, "sha1"))
print("SHA256:", compute_hash(sample, "sha256"))



MD5:    c3e77b6959cc68baee9825c84dc41d9c
SHA1:   bc18a67ad4057dd36f896a4d411b8fc5b06e5b2f
SHA256: 3b7ea4318c3c1508701102cf966f650e04f28d29938f85d74ec0ec2528657b6e


In [7]:
import re

def extract_strings(path):
    with open(path, "rb") as f:
        data = f.read()
    pattern = rb"[ -~]{4,}"   # matches printable ASCII strings of length ≥4
    return re.findall(pattern, data)

# Example usage:
strings = extract_strings(sample)

for s in strings[:20]:
    print(s.decode(errors="ignore"))



!This program cannot be run in DOS mode.
V*0T
0RichU
.text
`.rdata
@.data
.rsrc
@.reloc
hpqQ
h`EN
h|nN
h\nN
hlnN
=UUU
h_rM
hDLN
h`GO
hDLN
h|GO
hDLN


In [14]:
import pefile

sample = r"C:\Users\aisha\Downloads\ProcessMonitor\Procmon.exe"

pe = pefile.PE(sample)

print("Entry Point:", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))
print("Image Base:", hex(pe.OPTIONAL_HEADER.ImageBase))
print("\nImported DLLs and functions:")

for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(" ", entry.dll.decode())
    for imp in entry.imports[:5]:
        print("     -", imp.name.decode() if imp.name else "None")





Entry Point: 0xa7f70
Image Base: 0x400000

Imported DLLs and functions:
  WS2_32.dll
     - getsockname
     - listen
     - recv
     - closesocket
     - socket
  VERSION.dll
     - GetFileVersionInfoW
     - VerQueryValueW
     - GetFileVersionInfoSizeW
  COMCTL32.dll
     - ImageList_ReplaceIcon
     - ImageList_SetBkColor
     - ImageList_AddMasked
     - ImageList_BeginDrag
     - ImageList_EndDrag
  FLTLIB.DLL
     - FilterSendMessage
     - FilterGetMessage
     - FilterReplyMessage
     - FilterConnectCommunicationPort
  KERNEL32.dll
     - AcquireSRWLockExclusive
     - AcquireSRWLockShared
     - InitializeSRWLock
     - GetSystemInfo
     - VerSetConditionMask
  USER32.dll
     - LoadStringA
     - DrawEdge
     - GetMessageW
     - TranslateMessage
     - DispatchMessageW
  GDI32.dll
     - SaveDC
     - RestoreDC
     - SetBrushOrgEx
     - SetPixel
     - PatBlt
  COMDLG32.dll
     - ChooseColorW
     - GetOpenFileNameW
     - PrintDlgW
     - ChooseFontW
     - FindText

In [15]:
import yara

rule_source = """
rule ContainsHTTP {
    strings:
        $s = "http"
    condition:
        $s
}
"""

rules = yara.compile(source=rule_source)
matches = rules.match(sample)

print("YARA matches:", matches)


YARA matches: [ContainsHTTP]
